In [2]:
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm
import pandas as pd
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from keras.applications import DenseNet121

In [3]:
densenet = DenseNet121(
  weights='C:/Users/Bangik/Documents/python/diabetic-retinopathy-ml/model/DenseNet-BC-121-32-no-top.h5',
  include_top=False,
  input_shape=(224,224,3)
)

def build_model():
  model = Sequential()
  model.add(densenet)
  model.add(layers.GlobalAveragePooling2D())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(5, activation='sigmoid'))
  
  model.compile(
      loss='binary_crossentropy',
      optimizer=Adam(lr=0.00005),
      metrics=['accuracy']
  )
  
  return model

model = build_model()
model.load_weights('C:/Users/Bangik/Documents/python/diabetic-retinopathy-ml/model/model-cnn-17-5-23.h5')

c:\Users\Bangik\Documents\python\diabetic-retinopathy-ml\venv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
def crop_image_from_gray(img,tol=7):
  if img.ndim ==2:
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]
  elif img.ndim==3:
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = gray_img>tol        
    check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
    if (check_shape == 0):
      return img
    else:
      img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
      img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
      img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
      img = np.stack([img1,img2,img3],axis=-1)
      return img

def circle_crop_v2(img):
  img = crop_image_from_gray(img)

  height, width, depth = img.shape
  largest_side = np.max((height, width))
  img = cv2.resize(img, (largest_side, largest_side))

  height, width, depth = img.shape

  x = int(width / 2)
  y = int(height / 2)
  r = np.amin((x, y))

  circle_img = np.zeros((height, width), np.uint8)
  cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
  img = cv2.bitwise_and(img, img, mask=circle_img)
  img = crop_image_from_gray(img)

  return img

def preprocess_image(image_path, desired_size=224):
  img = cv2.imread(image_path)
  circle_crop = circle_crop_v2(img)
  bgr2rgb = cv2.cvtColor(circle_crop, cv2.COLOR_BGR2RGB)
  img_pil = Image.fromarray(bgr2rgb)
  im = img_pil.resize((desired_size, )*2, resample=Image.LANCZOS)
  return im

def prediction(image_path):
  tes_image = np.empty((1, 224, 224, 3), dtype=np.uint8)
  tes_image[0, :, :, :] = preprocess_image(image_path)
  predicted = model.predict(tes_image, verbose=None)
  probability = predicted.max()
  predicted = predicted > 0.5
  predicted = predicted.astype(int).sum(axis=1) - 1
  return predicted[0], probability

In [33]:
path_image = "E:/Pawang Code/Diabetic Retinopathy/Databases acquired by Service dOphtalmologie Hôpital Lariboisière Paris Messidor/Base11/20051020_43808_0100_PP.tif"
pil = Image.open(path_image).convert('RGB')
cv2image = np.array(pil)
tes = prediction(cv2image)
print(tes[0])

1/1 [==============================] - 6s 6s/step
0


In [8]:
train_df = pd.read_csv('E:/Pawang Code/Diabetic Retinopathy/Diabetic Retinopathy Detection competition kaggle/trainLabels.csv')
filename = []
y_true = []
y_pred = []

for i, image_id in enumerate(train_df.values):
  path_image = "E:/Pawang Code/Diabetic Retinopathy/Diabetic Retinopathy Detection competition kaggle/train/{}.jpeg".format(image_id[0])
  try:
    pil = Image.open(path_image).convert('RGB')
    cv2image = np.array(pil)
    tes = prediction(cv2image)
    filename.append(image_id[0])
    y_pred.append(tes[0])
    y_true.append(image_id[1])
  except:
    continue


1/1 [==============================] - 1s 1s/step


In [7]:
# list to csv
df = pd.DataFrame({'y_pred': y_pred})
df.to_csv('E:/Pawang Code/Diabetic Retinopathy/Diabetic Retinopathy Detection competition kaggle/result.csv', index=False)

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.95      0.86       567
           1       0.04      0.02      0.02        56
           2       0.31      0.10      0.15       110
           3       0.38      0.14      0.21        21
           4       0.40      0.14      0.21        14

    accuracy                           0.73       768
   macro avg       0.38      0.27      0.29       768
weighted avg       0.64      0.73      0.66       768

[[540  15  10   1   1]
 [ 54   1   1   0   0]
 [ 90   6  11   2   1]
 [  8   2   7   3   1]
 [  3   1   6   2   2]]


In [9]:
tes = prediction('E:/Pawang Code/Diabetic Retinopathy/INDIAN DIABETIC RETINOPATHY IMAGE DATASET/B. Disease Grading/1. Original Images/a. Training Set/IDRiD_001.jpg')
print(tes[0])

2


In [12]:
train_df = pd.read_csv('E:/Pawang Code/Diabetic Retinopathy/INDIAN DIABETIC RETINOPATHY IMAGE DATASET/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv')
dictDataframe = {'id_code': [], 'diagnosis': [], 'predicted': [], 'probability': []}

for i, image_id in enumerate(train_df.values):
    image_path = 'E:/Pawang Code/Diabetic Retinopathy/INDIAN DIABETIC RETINOPATHY IMAGE DATASET/B. Disease Grading/1. Original Images/b. Testing Set/{}.jpg'.format(image_id[0])
    tes = prediction(image_path)
    dictDataframe['id_code'].append(image_id[0])
    dictDataframe['diagnosis'].append(image_id[1])
    dictDataframe['predicted'].append(tes[0])
    dictDataframe['probability'].append(tes[1])
    print(i, image_id[0], tes[0], tes[1])

df = pd.DataFrame(dictDataframe)
df.to_csv('E:/Pawang Code/Diabetic Retinopathy/INDIAN DIABETIC RETINOPATHY IMAGE DATASET/B. Disease Grading/tes_prediction.csv', index=False)

0 IDRiD_001 3 0.9981096
1 IDRiD_002 1 0.9964221
2 IDRiD_003 1 0.9954413
3 IDRiD_004 3 0.99718165
4 IDRiD_005 2 0.99289644
5 IDRiD_006 2 0.99744606
6 IDRiD_007 2 0.99795616
7 IDRiD_008 2 0.99919456
8 IDRiD_009 2 0.999411
9 IDRiD_010 2 0.995725
10 IDRiD_011 2 0.9989399
11 IDRiD_012 1 0.99652314
12 IDRiD_013 2 0.99921304
13 IDRiD_014 2 0.9991358
14 IDRiD_015 2 0.9988653
15 IDRiD_016 3 0.9997142
16 IDRiD_017 2 0.9970253
17 IDRiD_018 2 0.9973249
18 IDRiD_019 2 0.99778783
19 IDRiD_020 2 0.99877656
20 IDRiD_021 2 0.9998134
21 IDRiD_022 2 0.9996836
22 IDRiD_023 2 0.9974227
23 IDRiD_024 1 0.99911135
24 IDRiD_025 3 0.9992489
25 IDRiD_026 4 0.9996484
26 IDRiD_027 2 0.9985191
27 IDRiD_028 2 0.9988945
28 IDRiD_029 1 0.9954122
29 IDRiD_030 1 0.99630266
30 IDRiD_031 2 0.99779373
31 IDRiD_032 2 0.9977945
32 IDRiD_033 2 0.99708855
33 IDRiD_034 2 0.9994487
34 IDRiD_035 3 0.9997527
35 IDRiD_036 2 0.9982913
36 IDRiD_037 1 0.99827987
37 IDRiD_038 1 0.9974463
38 IDRiD_039 0 0.99884766
39 IDRiD_040 2 0.99938

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(df['diagnosis'], df['predicted']))
print(confusion_matrix(df['diagnosis'], df['predicted']))

              precision    recall  f1-score   support

           0       0.70      0.21      0.32        34
           1       0.10      0.80      0.18         5
           2       0.43      0.56      0.49        32
           3       0.45      0.26      0.33        19
           4       0.00      0.00      0.00        13

    accuracy                           0.33       103
   macro avg       0.34      0.37      0.26       103
weighted avg       0.45      0.33      0.33       103

[[ 7 21  6  0  0]
 [ 1  4  0  0  0]
 [ 2 10 18  2  0]
 [ 0  2 11  5  1]
 [ 0  2  7  4  0]]
